In [13]:
library(BRVM)
library(IRkernel)
library(IRdisplay)
library(tidyverse)
library(tibble)
library(repr)
library(dplyr)
library(formattable)
library(scales)
library(readr)
library(rvest)
library(openxlsx)
library(skimr)

In [12]:
install.packages("skimr")


The downloaded binary packages are in
	/var/folders/80/lqqh37s14jn1pbg0fb61rf5c0000gp/T//RtmpT7SYiC/downloaded_packages


In [2]:
# Extract BRVM Info with BRVM_ticker_desc
brvm_info <- BRVM_ticker_desc()

# Extract tickers from ticker_info
tickers <- brvm_info$Ticker

In [3]:
# Retrieve BRVM information for the specified tickers
# from January 1st, 2023, to yesterday's date
brvm_info_list <- BRVM_get(.symbol = tickers, .from = "2023-01-01", .to = Sys.Date() - 1)

ABJCBICCBNBCBOABBOABFBOACBOAMBOANBOASCABCCBIBFCFACCIECECOCETITFTSCNEICNSBCNTLCONTBFORACORGTPALCPRSCSAFCSCRCSDCCSDSCSEMCSGBCSHECSIBCSICCSIVCSLBCSMBCSNTSSOGCSPHCSTACSTBCSVOCTTLCTTLSUNLCUNXC



In [4]:
# Merge the BRVM information retrieved for the specified tickers
# with additional BRVM information based on the common column "Ticker"
BRVM_frame <- merge(brvm_info_list, brvm_info, by = "Ticker")

In [6]:
# Display a message indicating the process of loading BRVM Market Cap information
cat("Loading BRVM Market Cap Info\n")

# Load BRVM market cap data
B_Market <- BRVM_cap()

# Rename the columns
B_Market <- B_Market %>%
  rename(Ticker = Symbol,
         `Market Cap` = `Global capitalization`,
         `Market Cap (%)` = `Global capitalization (%)`) %>%
  select(Ticker, `Market Cap`, `Market Cap (%)`)

# Format the "Market Cap" column as currency
B_Market$`Market Cap` <- currency(B_Market$`Market Cap`)

# Sort the data frame by "Market Cap" column in descending order
B_Market <- arrange(B_Market, desc(`Market Cap`))

# Format the "Market Cap (%)" column as percentage
B_Market$`Market Cap (%)` <- as.numeric(B_Market$`Market Cap (%)`)

# Print the modified data
print(B_Market)

Loading BRVM Market Cap Info
# A tibble: 47 x 3
   Ticker `Market Cap`      `Market Cap (%)`
   <chr>  <formttbl>                   <dbl>
 1 SNTS   1,781,000,000,000            22.6 
 2 ORAC   1,619,545,012,500            20.5 
 3 SGBC   550,666,647,000               6.98
 4 ECOC   372,967,815,000               4.73
 5 ETIT   325,513,920,996               4.13
 6 CBIBF  281,600,000,000               3.57
 7 SIBC   280,000,000,000               3.55
 8 ORGT   159,307,496,145               2.02
 9 BOABF  158,400,000,000               2.01
10 NTLC   149,416,608,000               1.9 
# i 37 more rows


In [24]:
# Merge BRVM dataframes by Ticker
BRVM_columns <- merge(BRVM_frame, B_Market, by = "Ticker") %>%

# Rename Company name column to Company
rename(Company = `Company name`) %>%

# Select relevant columns for BRVM analysis
select(Ticker,Sector, Company, Country, Date, Open, High, Low, Close, Volume, `Market Cap`, `Market Cap (%)`) %>%

# Arrange data by Market Cap in descending order
arrange(desc(`Market Cap`))

glimpse(BRVM_columns)

Rows: 12,408
Columns: 12
$ Ticker           <chr> "SNTS", "SNTS", "SNTS", "SNTS", "SNTS", "SNTS", "SNTS~
$ Sector           <fct> PUBLIC SERVICE, PUBLIC SERVICE, PUBLIC SERVICE, PUBLI~
$ Company          <chr> "SONATEL SENEGAL", "SONATEL SENEGAL", "SONATEL SENEGA~
$ Country          <fct> SENEGAL, SENEGAL, SENEGAL, SENEGAL, SENEGAL, SENEGAL,~
$ Date             <date> 2023-01-02, 2023-01-03, 2023-01-04, 2023-01-05, 2023~
$ Open             <dbl> 15195, 15390, 15295, 15265, 15100, 14990, 14600, 1500~
$ High             <dbl> 15395, 15390, 15295, 15265, 15100, 15000, 14800, 1500~
$ Low              <dbl> 15195, 15375, 15280, 15160, 15000, 14990, 14600, 1480~
$ Close            <dbl> 15395, 15375, 15280, 15160, 15000, 15000, 14800, 1480~
$ Volume           <dbl> 13037, 242, 14371, 3489, 813, 8148, 24572, 769, 2585,~
$ `Market Cap`     <formttbl> 1,781,000,000,000, 1,781,000,000,000, 1,781,000,~
$ `Market Cap (%)` <dbl> 22.59, 22.59, 22.59, 22.59, 22.59, 22.59, 22.59, 22.5~


In [25]:
# Sort dataframe in ascending order by a specific column
BRVM_columns_asc <- arrange(BRVM_columns, Date)

# Sort dataframe in descending order by a specific column
BRVM_columns_desc <- arrange(BRVM_columns, desc(Date))

In [26]:
# Sort dataframe in descending 
BRVM_columns_desc

Ticker,Sector,Company,Country,Date,Open,High,Low,Close,Volume,Market Cap,Market Cap (%)
<chr>,<fct>,<chr>,<fct>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<formttbl>,<dbl>
SNTS,PUBLIC SERVICE,SONATEL SENEGAL,SENEGAL,2024-03-15,17895,17895,17800,17810,9023,"1,781,000,000,000",22.59
ORAC,PUBLIC SERVICE,ORANGE COTE D'IVOIRE,IVORY COAST,2024-03-15,10760,10750,10750,10750,452,"1,619,545,012,500",20.54
SGBC,FINANCE,SOCIETE GENERALE COTE D'IVOIRE,IVORY COAST,2024-03-15,17650,17700,17650,17700,1859,"550,666,647,000",6.98
ECOC,FINANCE,ECOBANK COTE D'IVOIRE,IVORY COAST,2024-03-15,6775,6750,6750,6775,587,"372,967,815,000",4.73
ETIT,FINANCE,Ecobank Transnational Incorporated TOGO,TOGO,2024-03-15,18,18,18,18,353,"325,513,920,996",4.13
CBIBF,FINANCE,CORIS BANK INTERNATIONAL BURKINA FASO,BURKINA FASO,2024-03-15,8700,8800,8650,8800,77,"281,600,000,000",3.57
SIBC,FINANCE,SOCIETE IVOIRIENNE DE BANQUE COTE D'IVOIRE,IVORY COAST,2024-03-15,5700,5750,5600,5600,3099,"280,000,000,000",3.55
ORGT,FINANCE,ORAGROUP TOGO,TOGO,2024-03-15,2295,2150,2150,2295,10,"159,307,496,145",2.02
BOABF,FINANCE,BANK OF AFRICA BURKINA FASO,BURKINA FASO,2024-03-15,7200,7300,7300,7200,767,"158,400,000,000",2.01


In [27]:
# Select the first 40 rows of BRVM_columns_desc to create Realtime_BRVM
Realtime_BRVM <- head(BRVM_columns_desc, 40)

# Print a message indicating the display of Realtime BRVM Data Columns
cat('Realtime BRVM Data Columns')

# Print the Realtime_BRVM data frame
Realtime_BRVM

Realtime BRVM Data Columns

,Ticker,Sector,Company,Country,Date,Open,High,Low,Close,Volume,Market Cap,Market Cap (%)
,<chr>,<fct>,<chr>,<fct>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<formttbl>,<dbl>
1,SNTS,PUBLIC SERVICE,SONATEL SENEGAL,SENEGAL,2024-03-15,17895,17895,17800,17810,9023,"1,781,000,000,000",22.59
2,ORAC,PUBLIC SERVICE,ORANGE COTE D'IVOIRE,IVORY COAST,2024-03-15,10760,10750,10750,10750,452,"1,619,545,012,500",20.54
3,SGBC,FINANCE,SOCIETE GENERALE COTE D'IVOIRE,IVORY COAST,2024-03-15,17650,17700,17650,17700,1859,"550,666,647,000",6.98
4,ECOC,FINANCE,ECOBANK COTE D'IVOIRE,IVORY COAST,2024-03-15,6775,6750,6750,6775,587,"372,967,815,000",4.73
5,ETIT,FINANCE,Ecobank Transnational Incorporated TOGO,TOGO,2024-03-15,18,18,18,18,353,"325,513,920,996",4.13
6,CBIBF,FINANCE,CORIS BANK INTERNATIONAL BURKINA FASO,BURKINA FASO,2024-03-15,8700,8800,8650,8800,77,"281,600,000,000",3.57
7,SIBC,FINANCE,SOCIETE IVOIRIENNE DE BANQUE COTE D'IVOIRE,IVORY COAST,2024-03-15,5700,5750,5600,5600,3099,"280,000,000,000",3.55
8,ORGT,FINANCE,ORAGROUP TOGO,TOGO,2024-03-15,2295,2150,2150,2295,10,"159,307,496,145",2.02
9,BOABF,FINANCE,BANK OF AFRICA BURKINA FASO,BURKINA FASO,2024-03-15,7200,7300,7300,7200,767,"158,400,000,000",2.01


In [28]:
# Retrieve BRVM index data and store it in brvm_sector
brvm_sector <- BRVM_index()

# Rename the column "Indexes" to "Sector" in the brvm_sector dataframe
brvm_sector <- brvm_sector %>%
  rename(Sector = Indexes)

# Print a message indicating that BRVM Indexes are being loaded
print("Loading BRVM Indexes")

# Print the brvm_sector dataframe
brvm_sector

[1] "Loading BRVM Indexes"


Sector,Previous closing,Closing,Change (%),Year to Date Change
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
BRVM-30,106.84,106.59,-0.23,-0.36
BRVM - AGRICULTURE,162.38,163.42,0.64,2.09
BRVM - OTHER SECTOR,931.58,993.28,6.62,0.00
BRVM - COMPOSITE,212.07,211.92,-0.07,-0.36
BRVM - DISTRIBUTION,302.76,298.72,-1.33,1.12
BRVM - FINANCE,85.98,86.52,0.63,0.45
BRVM - INDUSTRY,100.79,100.95,0.16,-0.65
BRVM - PRESTIGE,99.13,98.83,-0.30,1.05
BRVM - PRINCIPAL,102.79,102.92,0.13,-1.06


In [30]:
# Exporting DataFrame into an Excel file for BRVM Sectors

# Write the DataFrame to an Excel file

write.xlsx(brvm_sector, "/Users/Phases/Desktop/BRVM Project/BRVM-Analyzer/BRVM Excel R files/BRVM_Sectors.xlsx", overwrite = TRUE)

In [31]:
# Convert the Date column to character type
Realtime_BRVM$Date <- as.character(Realtime_BRVM$Date)

# Export the DataFrame to an Excel file
write.xlsx(Realtime_BRVM, "/Users/Phases/Desktop/BRVM Project/BRVM-Analyzer/BRVM Excel R files/Realtime_BRVM.xlsx", overwrite = TRUE)

In [41]:
# Define the file path for the CSV file
file_path <- "/Users/Phases/Desktop/BRVM Project/BRVM-Analyzer/BRVM Tableau R files/Tableau_BRVM.csv"

# Check if the file already exists
if (file.exists(file_path)) {
  # If the file exists, remove it
  file.remove(file_path)
}

# Write the new CSV file
write.csv(BRVM_columns_desc, file = file_path, row.names = FALSE)

[1] TRUE

In [42]:
# Define the file path for the CSV file
file_path <- "/Users/Phases/Desktop/BRVM Project/BRVM-Analyzer/BRVM Tableau R files/Tableau_BRVM_Sectors.csv"

# Check if the file already exists
if (file.exists(file_path)) {
  # If the file exists, remove it
  file.remove(file_path)
}

# Write the new CSV file
write.csv(brvm_sector, file = file_path, row.names = FALSE)